<a href="https://colab.research.google.com/github/gduncan2/CS445_OpticalFlowProj/blob/main/445FinalProjCOODIES.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import cv2
import cupy as cp
import numpy as np
import glob
import os
from scipy.sparse import csr_matrix
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
def get_frame_pairs(frames: np.ndarray):
    return [(frames[i], frames[i + 1]) for i in range(0, len(frames), 2)]

def process_window_lk(
    window_size_x: int,
    window_size_y: int,
    start_x: int,
    start_y: int,
    frame_tplus1: np.ndarray,
    frame_t: np.ndarray,
    fps: float,
    epsilon: float = 1e-4
) -> cp.ndarray:
    # Calculate the temporal derivative
    # frame_t = cv2.cvtColor(frame_t, cv2.COLOR_BGR2RGB)
    # frame_tplus1 = cv2.cvtColor(frame_tplus1, cv2.COLOR_BGR2RGB)
    H, W, _ = frame_t.shape

    delta_t = 1 / fps
    # Patch extraction and GPU transfer
    patch_t = cp.asarray(frame_t[start_y:start_y+window_size_y, start_x:start_x+window_size_x].astype(np.float32))
    patch_t1 = cp.asarray(frame_tplus1[start_y:start_y+window_size_y, start_x:start_x+window_size_x].astype(np.float32))

    # Convert to grayscale
    patch_t_gray  = 0.2989 * patch_t[:,:,0] + 0.5870 * patch_t[:,:,1] + 0.1140 * patch_t[:,:,2]
    patch_t1_gray = 0.2989 * patch_t1[:,:,0] + 0.5870 * patch_t1[:,:,1] + 0.1140 * patch_t1[:,:,2]

    # Compute temporal derivative
    I_t = (patch_t1_gray - patch_t_gray) / delta_t

    # Placeholder for the spatial derivatives
    I_x = cp.zeros((window_size_x, window_size_y), dtype=cp.float32)
    I_y = cp.zeros((window_size_x, window_size_y), dtype=cp.float32)

    if start_x + window_size_x > W or start_y + window_size_y > H:
        return cp.zeros((2,1), dtype=cp.float32)

    smoothed = cp.asarray(cv2.GaussianBlur(cp.asnumpy(patch_t_gray), (5, 5), sigmaX=1.0, sigmaY=1.0))
    I_x = cp.asarray(cv2.Sobel(cp.asnumpy(smoothed), cv2.CV_32F, 1, 0, ksize=3) * 0.5) # ∂/∂x
    I_y = cp.asarray(cv2.Sobel(cp.asnumpy(smoothed), cv2.CV_32F, 0, 1, ksize=3) * 0.5) # ∂/∂y

    Ix = I_x.flatten()
    Iy = I_y.flatten()
    It = I_t.flatten()

    # The system of equations would look like this
    # [
    #   [∑ I_x^2,   ∑ I_x I_y],
    #   [∑ I_x I_y, ∑ I_y^2]
    # ]

    A00 = cp.sum(Ix * Ix)   # ∑ I_x^2
    A01 = cp.sum(Ix * Iy)   # ∑ I_x I_y
    A11 = cp.sum(Iy * Iy)   # ∑ I_y^2

    B0 = -cp.sum(Ix * It)   # -∑ I_x I_t
    B1 = -cp.sum(Iy * It)   # -∑ I_y I_t

    A = cp.array([[A00, A01],
                  [A01, A11]], dtype=cp.float32)
    B = cp.array([B0, B1],      dtype=cp.float32).reshape(2,1)

    # This is striaght from the video where we check how invertible the matrix is
    # In other words whether the system of equations are well conditioned or not
    # We can think of a system where there is hardly any change, like a patch of texture with no change
    det = A00*A11 - A01*A01
    if det > epsilon:
        uv = cp.linalg.solve(A, B)   # 2×1
    else:
        uv = cp.zeros((2,1), dtype=cp.float32)
    return uv



In [5]:
!ls "/content/drive/MyDrive/CS445/FinalProj/subset_of_frames"

In [3]:
folder = '/content/drive/MyDrive/CS445/FinalProj/'
frame_files = sorted(glob.glob(folder + 'subset_of_frames/frame_*.png'))
frames = []
for f in frame_files:
    img = cv2.imread(f)
    if img is not None:
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        frames.append(img_rgb)

if len(frames) == 0:
    raise RuntimeError("No frames loaded — check your folder path or file naming.")
fps = 24
window_w, window_h = 100, 100

frames = cp.array(frames)
T = len(frames)
H, W = frames[0].shape[:2]
nY = H // window_h
nX = W // window_w

flows = cp.zeros((T - 1, nY, nX, 2), dtype=cp.float32)

for t in range(T - 1):
    ft, ftplus1 = frames[t], frames[t + 1]
    for y in range(nY):
        y0 = y * window_h
        for x in range(nX):
            x0 = x * window_w
            uv = process_window_lk(window_w, window_h, start_x=x0, start_y=y0,
                                   frame_tplus1=ftplus1, frame_t=ft, fps=fps)
            flows[t, y, x, 0] = uv[0, 0]
            flows[t, y, x, 1] = uv[1, 0]

T, patch_y, patch_x, uv_shape = flows.shape
print(cp.asnumpy(flows).shape)

(7, 10, 19, 2)
